In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import dash_table

In [2]:
df = pd.read_csv("Top_Countries.csv")

In [3]:
df.head()

,id,country,iso_alpha,weight_kg,trade_usd
0,CHN,China,CHN,5.290000e+13,9.140000e+13
1,USA,United States of America,USA,5.750000e+12,7.330000e+13
2,DEU,Germany,DEU,1.320000e+13,5.870000e+13
3,JPN,Japan,JPN,1.990000e+13,3.720000e+13
4,FRA,France,FRA,8.280000e+12,2.940000e+13


In [4]:
df['id'] = df['iso_alpha']
df.set_index('id', inplace=True, drop=False)
print(df.columns)

Index(['id', 'country', 'iso_alpha', 'weight_kg', 'trade_usd'], dtype='object')


In [5]:
# App layout
app = JupyterDash(__name__, prevent_initial_callbacks=True)

In [6]:
app.layout = html.Div(children=[
    html.H3(children='Countries dominating World Trade',style={
            'textAlign': 'center',
            'color': '#3300D9'
        }),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True, "hideable": True}
            if i == "iso_alpha" or i == "id"
            else {"name": i, "id": i, "deletable": True, "selectable": True}
            for i in df.columns
        ],
        data=df.to_dict('records'),  # the contents of the table
        editable=True,              # allow editing of data inside all cells
        filter_action="native",     # allow filtering of data by user ('native') or not ('none')
        sort_action="native",       # enables data to be sorted per-column by user or not ('none')
        sort_mode="single",         # sort across 'multi' or 'single' columns
        column_selectable="multi",  # allow users to select 'multi' or 'single' columns
        row_selectable="multi",     # allow users to select 'multi' or 'single' rows
        row_deletable=True,         # choose if user can delete a row (True) or not (False)
        selected_columns=[],        # ids of columns that user selects
        selected_rows=[],           # indices of rows that user selects
        page_action="native",       # all data is passed to the table up-front or not ('none')
        page_current=0,             # page number that user is on
        page_size=6,                # number of rows visible per page
        style_cell={                # ensure adequate header width when text is shorter than cell's text
            'minWidth': 95, 'maxWidth': 95, 'width': 95
        },
        style_cell_conditional=[    # align text columns to left. By default they are aligned to right
            {
                'if': {'column_id': c},
                'textAlign': 'left'
            } for c in ['country']
        ],
        style_data={                # overflow cells' content into multiple lines
            'whiteSpace': 'normal',
            'height': 'auto'
        }
    ),
    
    html.Br(),
    html.Br(),
    html.Div(id='bar-container'),
    html.Div(id='choromap-container')

])


In [7]:
# Create bar chart
@app.callback(
    Output(component_id='bar-container', component_property='children'),
    [Input(component_id='datatable-interactivity', component_property="derived_virtual_data"),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_selected_rows'),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_selected_row_ids'),
     Input(component_id='datatable-interactivity', component_property='selected_rows'),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_indices'),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_row_ids'),
     Input(component_id='datatable-interactivity', component_property='active_cell'),
     Input(component_id='datatable-interactivity', component_property='selected_cells')]
)
def update_bar(all_rows_data, slctd_row_indices, slct_rows_names, slctd_rows,
               order_of_rows_indices, order_of_rows_names, actv_cell, slctd_cell):
    print('***************************************************************************')
    print('Data across all pages pre or post filtering: {}'.format(all_rows_data))
    print('---------------------------------------------')
    print("Indices of selected rows if part of table after filtering:{}".format(slctd_row_indices))
    print("Names of selected rows if part of table after filtering: {}".format(slct_rows_names))
    print("Indices of selected rows regardless of filtering results: {}".format(slctd_rows))
    print('---------------------------------------------')
    print("Indices of all rows pre or post filtering: {}".format(order_of_rows_indices))
    print("Names of all rows pre or post filtering: {}".format(order_of_rows_names))
    print("---------------------------------------------")
    print("Complete data of active cell: {}".format(actv_cell))
    print("Complete data of all selected cells: {}".format(slctd_cell))

    dff = pd.DataFrame(all_rows_data)

    # used to highlight selected countries on bar chart
    colors = ['#7FDBFF' if i in slctd_row_indices else '#D99800'
              for i in range(len(dff))]

    if "country" in dff and "trade_usd" in dff:
        return [
            dcc.Graph(id='bar-chart',
                      figure=px.bar(
                          data_frame=dff,
                          x="country",
                          y='trade_usd',
                          labels={"trade_usd": "Total amount traded"}
                      ).update_layout(showlegend=False, xaxis={'categoryorder': 'total ascending'})
                      .update_traces(marker_color=colors, hovertemplate="<b>%{y}%</b><extra></extra>")
                      )
        ]


In [8]:
# Create choropleth map
@app.callback(
    Output(component_id='choromap-container', component_property='children'),
    [Input(component_id='datatable-interactivity', component_property="derived_virtual_data"),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_selected_rows')]
)
def update_map(all_rows_data, slctd_row_indices):
    dff = pd.DataFrame(all_rows_data)

    # highlight selected countries on map
    borders = [5 if i in slctd_row_indices else 1
               for i in range(len(dff))]

    if "iso_alpha" in dff and "trade_usd" in dff and "country" in dff:
        return [
            dcc.Graph(id='choropleth',
                      style={'height': 700},
                      figure=px.choropleth(
                          data_frame=dff,
                          locations="iso_alpha",
                          scope="world",
                          color="trade_usd",
                          title="Total amount traded",
                          template='plotly_dark',
                          hover_data=['country', 'trade_usd'],
                      ).update_layout(showlegend=False, title=dict(font=dict(size=28), x=0.5, xanchor='center'))
                      .update_traces(marker_line_width=borders, hovertemplate="<b>%{customdata[0]}</b><br><br>" +
                                                                              "%{customdata[1]}")
                      )
        ]

In [9]:
# Highlight selected column
@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

In [10]:
app.run_server(mode='inline')

***************************************************************************
Data across all pages pre or post filtering: [{'id': 'CHN', 'country': 'China', 'iso_alpha': 'CHN', 'weight_kg': 52900000000000, 'trade_usd': 91400000000000}, {'id': 'USA', 'country': 'United States of America', 'iso_alpha': 'USA', 'weight_kg': 5750000000000, 'trade_usd': 73300000000000}, {'id': 'DEU', 'country': 'Germany', 'iso_alpha': 'DEU', 'weight_kg': 13200000000000, 'trade_usd': 58700000000000}, {'id': 'JPN', 'country': 'Japan', 'iso_alpha': 'JPN', 'weight_kg': 19900000000000, 'trade_usd': 37200000000000}, {'id': 'FRA', 'country': 'France', 'iso_alpha': 'FRA', 'weight_kg': 8280000000000, 'trade_usd': 29400000000000}, {'id': 'CAN', 'country': 'Canada', 'iso_alpha': 'CAN', 'weight_kg': 12900000000000, 'trade_usd': 27600000000000}, {'id': 'BEL', 'country': 'Belgium', 'iso_alpha': 'BEL', 'weight_kg': 8780000000000, 'trade_usd': 23500000000000}, {'id': 'GBR', 'country': 'United Kingdom', 'iso_alpha': 'GBR', 'w